In [35]:
%matplotlib notebook
import numpy as numpy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [91]:
DEATH_BY_COV = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
CASES_COV = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

In [5]:
def data_preparation(url):
    """
    Function to prepare the data to be analyzed.
    """
    data_df = pd.read_csv(url)
    colums_exclud = data_df.columns[[0, 2, 3]]
    data_df.drop(colums_exclud,
                 axis=1,
                 inplace=True)
    data_df = data_df.set_index("Country/Region")
    data_df = data_df.groupby(level=0).sum()
    return data_df


In [107]:
def data_country(cases_table, death_table, country):
    data_cases = cases_table.loc[country]
    data_death = death_table.loc[country]
    data_array = np.c_[data_cases.index,
                       data_cases.values,
                       data_death.values]
    return data_array

In [110]:
data_country(cases_table=cases_df, death_table=death_df, country="Brazil")

array([['1/22/20', 0, 0],
       ['1/23/20', 0, 0],
       ['1/24/20', 0, 0],
       ['1/25/20', 0, 0],
       ['1/26/20', 0, 0],
       ['1/27/20', 0, 0],
       ['1/28/20', 0, 0],
       ['1/29/20', 0, 0],
       ['1/30/20', 0, 0],
       ['1/31/20', 0, 0],
       ['2/1/20', 0, 0],
       ['2/2/20', 0, 0],
       ['2/3/20', 0, 0],
       ['2/4/20', 0, 0],
       ['2/5/20', 0, 0],
       ['2/6/20', 0, 0],
       ['2/7/20', 0, 0],
       ['2/8/20', 0, 0],
       ['2/9/20', 0, 0],
       ['2/10/20', 0, 0],
       ['2/11/20', 0, 0],
       ['2/12/20', 0, 0],
       ['2/13/20', 0, 0],
       ['2/14/20', 0, 0],
       ['2/15/20', 0, 0],
       ['2/16/20', 0, 0],
       ['2/17/20', 0, 0],
       ['2/18/20', 0, 0],
       ['2/19/20', 0, 0],
       ['2/20/20', 0, 0],
       ['2/21/20', 0, 0],
       ['2/22/20', 0, 0],
       ['2/23/20', 0, 0],
       ['2/24/20', 0, 0],
       ['2/25/20', 0, 0],
       ['2/26/20', 1, 0],
       ['2/27/20', 1, 0],
       ['2/28/20', 1, 0],
       ['2/29/20', 2,

In [102]:
cases_df = data_preparation(url=CASES_COV)
death_df = data_preparation(url=DEATH_BY_COV)

In [105]:
death_df

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,1113,1147,1164,1181,1185,1186,1190,1211,1225,1248
Albania,0,0,0,0,0,0,0,0,0,0,...,104,107,111,112,113,117,120,123,128,134
Algeria,0,0,0,0,0,0,0,0,0,0,...,1052,1057,1068,1078,1087,1100,1111,1124,1136,1146
Andorra,0,0,0,0,0,0,0,0,0,0,...,52,52,52,52,52,52,52,52,52,52
Angola,0,0,0,0,0,0,0,0,0,0,...,28,29,29,29,29,30,33,33,35,39
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
Argentina,0,0,0,0,0,0,0,0,0,0,...,2112,2178,2220,2260,2373,2490,2588,2702,2807,2893
Armenia,0,0,0,0,0,0,0,0,0,0,...,607,620,631,641,650,662,678,688,692,700
Australia,0,0,0,0,0,0,0,0,0,0,...,116,118,122,123,126,128,133,139,145,155


In [106]:
cases_df

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,35070,35229,35301,35475,35526,35615,35727,35928,35981,36036
Albania,0,0,0,0,0,0,0,0,0,0,...,3851,3906,4008,4090,4171,4290,4358,4466,4570,4637
Algeria,0,0,0,0,0,0,0,0,0,0,...,21355,21948,22549,23084,23691,24278,24872,25484,26159,26764
Andorra,0,0,0,0,0,0,0,0,0,0,...,877,880,880,880,884,884,889,889,897,897
Angola,0,0,0,0,0,0,0,0,0,0,...,607,638,687,705,749,779,812,851,880,916
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,74,76,76,76,76,76,76,76,82,82
Argentina,0,0,0,0,0,0,0,0,0,0,...,114783,119301,122524,126755,130774,136118,141900,148027,153520,158334
Armenia,0,0,0,0,0,0,0,0,0,0,...,33559,34001,34462,34877,34981,35254,35693,36162,36613,36996
Australia,0,0,0,0,4,5,5,6,9,9,...,11233,11441,11802,12069,12428,12894,13302,13595,13950,14403
